<a href="https://colab.research.google.com/github/argennof/VisCompu2022/blob/main/Lab_5_Semantic_and_Instance_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Semantic Segmentation and Instance Segmentation

**NOTA**: Este notebook ha sido creado a partir de diferentes recursos disponibles en la web, los cuales se listan a continuación.
- https://d2l.ai/chapter_computer-vision/semantic-segmentation-and-dataset.html
- https://www.tensorflow.org/tutorials/images/segmentation?hl=en
- https://nanonets.com/blog/how-to-do-semantic-segmentation-using-deep-learning/
- https://nanonets.com/blog/semantic-image-segmentation-2020/
- https://towardsdatascience.com/review-fcn-semantic-segmentation-eb8c9b50d2d1

---




## Semantic Segmentation

La segmentación semántica intenta **segmentar imágenes** en regiones con diferentes **categorías semánticas**. Estas regiones semánticas se usan para etiquetar y predecir objetos a nivel de píxeles. La siguiente figura muestra una imagen segmentada semánticamente, con áreas etiquetadas para "Perro", "Gato", y "Background". 

![](https://d2l.ai/_images/segmentation.svg)

Por lo tanto, la segmentación semántica trata de descubrir con precisión el límite exacto de los objetos en la imagen diferenciandose del resto de las tareas más habituales de computer vision.


## Instance Segmentation

Por otro lado, la **segmentación de instancias** (también llamada detección y segmentación simultánea) se enfoca en reconocer las regiones a nivel de píxeles de cada instancia de objeto en una imagen. A diferencia de la segmentación semántica, la segmentación de instancias necesita distinguir no solo la semántica, sino también diferentes instancias de objetos. Por ejemplo, si hay dos perros en la imagen, la segmentación de instancias debe distinguir a cuál de los dos perros pertenece un píxel.

![](https://nanonets.com/blog/content/images/size/w1000/2020/08/59b6d0529299e.png)

# Semantic Segmentation con CNNs

En general, las CNNs diseñadas para la tarea de segmentación semántica presentan una arquitectura que constan de dos partes esenciales: un **encoder** y un **decoder**. 

![](https://www.mdpi.com/applsci/applsci-07-00312/article_deploy/html/images/applsci-07-00312-g002.png)

El **encoder** consiste en una serie de *capas convolucionales* y *capas de pooling* (o submuestreo) que se encargan de la extracción automática de features. Como ya vimos en notebooks anteriores, el propósito del pooling es reducir la resolución de los features maps a medida que se avanza en la red. Esto permite mejorar la robustez del clasificador, eliminando features redundantes y haciendo al algoritmo invariante espacialmente. Por lo tanto, el propósito principal del decoder es obtener una *representación abstracta* de baja resolución la imagen.

Por otro lado, el **decoder** se encarga de *proyectar semánticamente* los feature maps de baja resolución aprendidas por el encoder en una *máscara de segmentación*, esto es, una imagen de igual resolución que la imagen de entrada que representa la clasificación densa por píxeles. Dado que los features maps del encoder sufren una pérdida de resolución espacial, el decoder utiliza una serie de [*capas de upsampling*](https://www.oreilly.com/library/view/deep-learning-for/9781788295628/467cf02b-dc52-49c5-9289-b2721f6758da.xhtml) (lo contrario al pooling) y [*capas de convolución transpuesta*](https://www.oreilly.com/library/view/hands-on-convolutional-neural/9781789130331/c6c4a7a2-7776-454a-83c5-3779ab807a00.xhtml) (también llamadas de-convoluciones) para proyectar las features maps del encoder al espacio de píxeles. Por último, la *capa softmax* es la encargada de realizar la clasificación de píxeles. 

En este notebook vamos a utilizar una arquitectura (encoder y decoder) inspirada en las **[Fully Convolutional Networks (FCN)](https://arxiv.org/pdf/1411.4038.pdf)**.


## Fully Convolutional Network: de clasificación de imágenes a segmentación semántica.







Este turial se enfoca en la tarea de segmentación semántica, usando una versión modificada de la arquitectura [U-Net](https://lmb.informatik.uni-freiburg.de/people/ronneber/u-net/).

![](https://lmb.informatik.uni-freiburg.de/people/ronneber/u-net/u-net-architecture.png)

Vamos a instalar e importar las depedencias necesarias.

In [ ]:
!pip install git+https://github.com/tensorflow/examples.git

In [ ]:
import tensorflow as tf

import tensorflow_datasets as tfds

In [ ]:
from tensorflow_examples.models.pix2pix import pix2pix

from IPython.display import clear_output
import matplotlib.pyplot as plt

## Descargarmos el Oxford-IIIT Pets dataset

Este tutorial utiliza el [Oxford-IIIT Pet Dataset](https://www.robots.ox.ac.uk/~vgg/data/pets/) ([Parkhi et al, 2012](https://www.robots ox.ac.uk/~vgg/publications/2012/parkhi12a/parkhi12a.pdf)). Este dataset consta de imágenes de 37 razas de mascotas, con 200 imágenes por raza (~100 cada una en los splits de train y test). Cada imagen incluye las etiquetas correspondientes y las máscaras de píxeles. Las máscaras son etiquetas de clase para cada píxel. A cada píxel se le asigna una de tres categorías:

- Clase 1: Píxel perteneciente a la mascota.
- Clase 2: Píxel bordeando a la mascota.
- Clase 3: ninguna de las anteriores/un píxel circundante.

Este dataset esta disponibe en el repositorio de [TensorFlow Datasets](https://www.tensorflow.org/datasets/catalog/oxford_iiit_pet). Las máscaras de segmentación son incluídas en la versión 3+ del dataset.

In [ ]:
dataset, info = tfds.load('oxford_iiit_pet:3.*.*', with_info=True)

> **NOTA IMPORTANTE**: Las celdas que siguen a continuación definen algunas funciones utilitarias y establecen un pipeline para el preprocesamiento de las imágenes del dataset, para ajustarlo a los requerimientos del proceso de entrenamiento y validación del algoritmo.** Se dejan breves explicaciones sobre que hace cada celda, pero no es el objetivo de este notebook profundizar en esas temáticas**. Pueden simplemente asumir que que estás celdas hacen su trabajo y avanzar.

Primero vamos a definir una función para normalizar los datos y un función para cargar las imágenes.

La función `normalize()` normaliza los valores de color de la imagen al rango `[0, 1]`. Además, siendo que los píxeles de las máscaras de segmentación están etiquetados como clase {1, 2, 3}, por conveniencia restamos 1 a las mismas, lo que da como resultado etiquetas: {0, 1, 2}.

In [ ]:
def normalize(input_image, input_mask):
  input_image = tf.cast(input_image, tf.float32) / 255.0
  input_mask -= 1
  return input_image, input_mask

In [ ]:
def load_image(datapoint):
  input_image = tf.image.resize(datapoint['image'], (128, 128))
  input_mask = tf.image.resize(datapoint['segmentation_mask'], (128, 128))

  input_image, input_mask = normalize(input_image, input_mask)

  return input_image, input_mask

El dataset ya contiene los splits de para entrenamiento y testeo requeridos, por lo que usaremos esas mismas divisiones:

In [ ]:
TRAIN_LENGTH = info.splits['train'].num_examples
BATCH_SIZE = 64
BUFFER_SIZE = 1000
STEPS_PER_EPOCH = TRAIN_LENGTH // BATCH_SIZE

In [ ]:
train_images = dataset['train'].map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
test_images = dataset['test'].map(load_image, num_parallel_calls=tf.data.AUTOTUNE)

La siguiente clase realiza un proceso de *augmentations*, volteando aleatoriamente una imagen. En el tutorial [Image augmentation](https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/images/data_augmentation.ipynb) pueden obtener más información sobre este proceso.


In [ ]:
class Augment(tf.keras.layers.Layer):
  def __init__(self, seed=42):
    super().__init__()
    # both use the same seed, so they'll make the same random changes.
    self.augment_inputs = tf.keras.layers.RandomFlip(mode="horizontal", seed=seed)
    self.augment_labels = tf.keras.layers.RandomFlip(mode="horizontal", seed=seed)
  
  def call(self, inputs, labels):
    inputs = self.augment_inputs(inputs)
    labels = self.augment_labels(labels)
    return inputs, labels

Creamos un pipeline de entrada para las imágenes de entrenamiento y testeo:


In [ ]:
train_batches = (
    train_images
    .cache()
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE)
    .repeat()
    .map(Augment())
    .prefetch(buffer_size=tf.data.AUTOTUNE))

test_batches = test_images.batch(BATCH_SIZE)

Definimos una función para visualizar una imagen y su máscara correspondiente en el dataset:

In [ ]:
def display(display_list):
  plt.figure(figsize=(15, 15))

  title = ['Input Image', 'True Mask', 'Predicted Mask']

  for i in range(len(display_list)):
    plt.subplot(1, len(display_list), i+1)
    plt.title(title[i])
    plt.imshow(tf.keras.utils.array_to_img(display_list[i]))
    plt.axis('off')
  plt.show()

In [ ]:
for images, masks in train_batches.take(2):
  sample_image, sample_mask = images[0], masks[0]
  display([sample_image, sample_mask])

## El modelo

El modelo que se utiliza aquí es una [U-Net](https://arxiv.org/abs/1505.04597) modificado. Una U-Net consta de un encoder (downsampler) y un decoder (upsampler). Para aprender features sólidas y reducir la cantidad de parámetros entrenables, se usa como encoder un modelo previamente entrenado: [MobileNetV2](https://arxiv.org/abs/1801.04381). Para el decoder se utilizará el bloque *upsample* previamente implementado en el ejemplo [pix2pix](https://github.com/tensorflow/examples/blob/master/tensorflow_examples/models/pix2pix/pix2pix.py) en el repositorio de ejemplos de TensorFlow.


Para conseguir el modelo MobileNetV2 preentrenado vamos a utilizar `tf.keras.applications`, un módulo de TensorFlow que nos ofrece un amplio repositorio de diferentes arquitecturas con sus pesos preentrenados. Tener en cuenta que el encoder no se entrenará durante el proceso de entrenamiento, es decir, sus pesos no se van a actualizar.

In [ ]:
base_model = tf.keras.applications.MobileNetV2(input_shape=[128, 128, 3], include_top=False)

# Use the activations of these layers
layer_names = [
    'block_1_expand_relu',   # 64x64
    'block_3_expand_relu',   # 32x32
    'block_6_expand_relu',   # 16x16
    'block_13_expand_relu',  # 8x8
    'block_16_project',      # 4x4
]
base_model_outputs = [base_model.get_layer(name).output for name in layer_names]

# Create the feature extraction model
down_stack = tf.keras.Model(inputs=base_model.input, outputs=base_model_outputs)

down_stack.trainable = False

The decoder/upsampler is simply a series of upsample blocks implemented in TensorFlow examples:

El decoder/upsampler es simplemente una serie de bloques de muestreo ascendente (upsamples) implementados en los ejemplos de TensorFlow:

In [ ]:
up_stack = [
    pix2pix.upsample(512, 3),  # 4x4 -> 8x8
    pix2pix.upsample(256, 3),  # 8x8 -> 16x16
    pix2pix.upsample(128, 3),  # 16x16 -> 32x32
    pix2pix.upsample(64, 3),   # 32x32 -> 64x64
]

Finalmente ensamblamos todo en una sola arquitectura:

In [ ]:
def unet_model(output_channels:int):
  inputs = tf.keras.layers.Input(shape=[128, 128, 3])

  # Downsampling through the model
  skips = down_stack(inputs)
  x = skips[-1]
  skips = reversed(skips[:-1])

  # Upsampling and establishing the skip connections
  for up, skip in zip(up_stack, skips):
    x = up(x)
    concat = tf.keras.layers.Concatenate()
    x = concat([x, skip])

  # This is the last layer of the model
  last = tf.keras.layers.Conv2DTranspose(
      filters=output_channels, kernel_size=3, strides=2,
      padding='same')  #64x64 -> 128x128

  x = last(x)

  return tf.keras.Model(inputs=inputs, outputs=x)

Tenga en cuenta que la cantidad de filtros en la última capa se establece en la cantidad de `output_channels`. Este será un nodo de salida por clase.

## Entrenamiento del modelo

Ahora, todo lo que queda por hacer es compilar y entrenar el modelo.

Dado que este es un problema de clasificación multiclase, usamos la función de pérdida `tf.keras.losses.CategoricalCrossentropy` con el argumento `from_logits` establecido en `True`, ya que las etiquetas son números enteros escalares en lugar de vectores de puntajes para cada píxel de cada clase.

In [ ]:
OUTPUT_CLASSES = 3

model = unet_model(output_channels=OUTPUT_CLASSES)
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

Graficamos la arquitectura del modelo resultante:


In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True)

Al momento de realizar la inferencia, la etiqueta asignada a un píxel es el nodo (o canal) con el valor más alto. Esta asignación la realiza la función `create_mask()`.

La función `show_predictions()` muestra la imagen de entrada, la máscara groundtruth (i.e. la máscara etiquetada por un experto) y la predicción realizada por el modelo.

In [ ]:
def create_mask(pred_mask):
  pred_mask = tf.math.argmax(pred_mask, axis=-1)
  pred_mask = pred_mask[..., tf.newaxis]
  return pred_mask[0]

In [ ]:
def show_predictions(dataset=None, num=1):
  if dataset:
    for image, mask in dataset.take(num):
      pred_mask = model.predict(image)
      display([image[0], mask[0], create_mask(pred_mask)])
  else:
    display([sample_image, sample_mask,
             create_mask(model.predict(sample_image[tf.newaxis, ...]))])

Probamos el modelo para comprobar lo que predice antes del entrenamiento:

In [ ]:
show_predictions()

La función `DisplayCallback()` definida a continuación se usa para visualizar cómo se comporta la función de pérdida del modelo mientras se está entrenando:

In [ ]:
class DisplayCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    clear_output(wait=True)
    show_predictions()
    print ('\nSample Prediction after epoch {}\n'.format(epoch+1))

Ejecutamos el entrenamiento del modelo.

In [ ]:
EPOCHS = 20
VAL_SUBSPLITS = 5
VALIDATION_STEPS = info.splits['test'].num_examples//BATCH_SIZE//VAL_SUBSPLITS

model_history = model.fit(train_batches, epochs=EPOCHS,
                          steps_per_epoch=STEPS_PER_EPOCH,
                          validation_steps=VALIDATION_STEPS,
                          validation_data=test_batches,
                          callbacks=[DisplayCallback()])

En aras de ahorrar tiempo, el número de épocas se mantuvo pequeño, pero puede configurarlo más alto para lograr resultados más precisos.

In [ ]:
loss = model_history.history['loss']
val_loss = model_history.history['val_loss']

plt.figure()
plt.plot(model_history.epoch, loss, 'r', label='Training loss')
plt.plot(model_history.epoch, val_loss, 'bo', label='Validation loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss Value')
plt.ylim([0, 1])
plt.legend()
plt.show()

## Inferencia sobre nuevas imágenes

Ahora, vamos a hacer algunas predicciones.

In [ ]:
show_predictions(test_batches, 10)

---

# Trabajo Práctico 2 (segunda parte - Opcional)

**Acá tienen que dejar los datos de las y los integrantes del grupo:**

Nombre y Apellido, DNI, correo eletrónico

---

## **EJERCICIO 5.1**: segmentar nuevas imágenes de mascotas (perros y gatos) con el modelo previamente entrenado

1. **Crear un nuevo dataset** propio con un mínimo de **10 imágenes**, donde cada una contenga **al menos 2 mascotas** en escena. Pueden capturar imágenes de sus propias mascotas o buscarlas en la web. Por ejemplo:
![](https://assets.jumpseller.com/store/pets-center/themes/476474/options/69738368/flying-with-pets-og-image-1200x630-1.jpeg?1651702172)
2. **Predecir las máscaras de segmentación** de cada imagen.

TIP: reutilice las celdas de código presentadas anteriormente


In [ ]:
# EJERCICIO 1.1 (OPCIONAL)
# ...

## **EJERCICIO 5.2**: desarrollar un enfoque para la tarea de **instance segmentation**

A partir de las habilidades y conocimientos adquiridos en el Lab3 y 4, ahora es posible combinar el modelo de detección de objetos con el modelo de segmentación semántica para desarrollar un enfoque de segmentación de instancias. Por ejemplo, usar un modelo para detectar perros y gatos en una imágen y sobre la región de cada detección aplicar un modelo de segmentación (o viceversa). La combinación de ambas produce un segmentación de instancias. Estrictamente hablando, este enfoque no es un modelo por si mismo, sino la concatenación de dos modelos diferentes.

Una alternativa es utilizar un único modelo cuya arquitectura fue pensada para la tarea de segmentación de instancias. Existen una gran variedad de modelos que pueden reutilizar para esto. Uno de las más conocidas en Mask R-CNN. En [este artículo](https://towardsdatascience.com/computer-vision-instance-segmentation-with-mask-r-cnn-7983502fcad1) se explora Mask R-CNN para comprender su funcionamiento y como implementarlo usando Keras.



In [ ]:
# EJERCICIO 1.2 (OPCIONAL)
# ...